# Implementing Laplace for $\epsilon$-DP

## Generate Data

We'll create a dummy dataset with some records of fake people who we can then summarize in a some tables.

The characteristics we'll use will be:
* Sex
* Age
* Marital Status
* Earnings
* Occupation

In [1]:
# Some initial Python setup requirements for later code.

# Required to get the plots inline for Census implementation.
%matplotlib inline

# Load the libraries we need.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import truncnorm

# Set the threshold for numpy output values that get printed to the screen.
np.set_printoptions(threshold=10)
pd.options.display.max_rows = 10

In [2]:
# Specify how many dummy records we want.
numrecs=100

# Valid Values
sexvalid=["M", "F"]
marrvalid=["M", "W", "S", "D", "N"]
occvalid=["Statistician", "Economist", "Geographer", "IT Specialist", "Unicorn Wrangler"]

# Generate arrays of random values.
sexarray = np.random.choice(a=sexvalid,size=numrecs,p=[0.50, 0.50])
agearray = np.random.normal(loc=45, scale=10, size=numrecs).round().astype(np.int)
marrarray = np.random.choice(a=marrvalid,size=numrecs,p=[0.478, 0.057,0.109,0.019,0.337])
earnarray=np.power(10,truncnorm.rvs(-3, 4, loc=5, scale=0.5, size=numrecs)).round().astype(np.int)
occarray=np.random.choice(a=occvalid,size=numrecs,p=[0.3,0.3,0.2,0.19,0.01])


#myrecs=pd.DataFrame(np.array([sexarray,agearray]))
myrecs = pd.DataFrame({'Sex': sexarray, 
                       'Age': agearray,
                       'Married': marrarray,
                       'Earnings': earnarray,
                       'Occupation': occarray
                    }, columns=['Sex', 'Age', 'Married', 'Earnings', 'Occupation'])
print(myrecs)


   Sex  Age Married  Earnings     Occupation
0    F   47       N    124539     Geographer
1    M   51       N     34158   Statistician
2    F   58       M    272855   Statistician
3    F   54       M    125639   Statistician
4    M   53       W    225023   Statistician
..  ..  ...     ...       ...            ...
95   M   37       M     13114   Statistician
96   M   54       N     50333   Statistician
97   M   35       N    168241   Statistician
98   M   50       N    126798  IT Specialist
99   M   40       N    363350   Statistician

[100 rows x 5 columns]


## Summarize Data
For this example, we want to publish a count of Occupation by Sex. For reference, we'll start by looking at the true version of the table.

In [3]:
# Note that we reindex using the full set of valid values. Unless we have structural zeros, we must also protect the real zeros.
truetab=pd.crosstab(index=myrecs['Occupation'], columns=myrecs['Sex'])
truetab=truetab.reindex(index=occvalid, columns=sexvalid, fill_value=0)
print(truetab)


Sex                M   F
Occupation              
Statistician      17  17
Economist         12  13
Geographer         9   8
IT Specialist     10  12
Unicorn Wrangler   0   2


# Generate Laplace Noise
Now we will generate a noise value for each cell in the table too be published.

In [11]:
# First let's define the scale of the Laplace distribution in terms of sensitivity and epsilon.
sensitivity=1.
epsilon=1.
scale=sensitivity/epsilon

print("Epsilon = {:4.2f}".format(epsilon))
print("Sensitivity = {:4.2f}".format(sensitivity) )
print("Scale = {:4.2f}".format(scale))
print()


noise = np.random.laplace(0,scale,truetab.shape)
print(noise)


Epsilon = 1.00
Sensitivity = 1.00
Scale = 1.00

[[ 1.3933897  -0.09805214]
 [ 0.37031732  0.26280791]
 [ 0.45743449 -0.63277477]
 [ 0.87986425  0.4714722 ]
 [ 0.32439922 -0.51915892]]


# Create Noisy Table
Now we add true data and the noise to get a noise table. Depending upon our publication strategy we may want to post-process the table (e.g. rounding, excluding negative values).

In [12]:
# Print the noisy table.
noisytab=truetab+noise
print(noisytab)
print()

# Post-process the noisy table to round and set negative values to zero.
noisytabpost=noisytab.round().astype(np.int)
noisytabpost[noisytabpost < 0]=0
print("Noisy table to be published:")
print(noisytabpost)
print()

print("True table for comparison:")
print(truetab)

Sex                       M          F
Occupation                            
Statistician      18.393390  16.901948
Economist         12.370317  13.262808
Geographer         9.457434   7.367225
IT Specialist     10.879864  12.471472
Unicorn Wrangler   0.324399   1.480841

Noisy table to be published:
Sex                M   F
Occupation              
Statistician      18  17
Economist         12  13
Geographer         9   7
IT Specialist     11  12
Unicorn Wrangler   0   1

True table for comparison:
Sex                M   F
Occupation              
Statistician      17  17
Economist         12  13
Geographer         9   8
IT Specialist     10  12
Unicorn Wrangler   0   2


# Measure the Error
Now we will calculate the $L_{1}$ error between the true table and the noisy table. First cell by cell and then in total.

In [13]:
# First cell-by-cell errors.
errortab=np.abs(noisytabpost-truetab)
print("Cell-by-cell absolute error:")
print(errortab)
print()

# Then percentage errors.
pcterrortab=errortab/truetab
print("Cell-by-cell relative error:")
print(pcterrortab)
print()

# Now the total error and the relative error.
errorsum=errortab.sum().sum()
print("Total L1 Error: {0:1d}".format(errorsum))
truesum=truetab.sum().sum()
errorrel=100*errorsum/truesum
print("Relative L1 Error: {:4.2f}%".format(errorrel))

Cell-by-cell absolute error:
Sex               M  F
Occupation            
Statistician      1  0
Economist         0  0
Geographer        0  1
IT Specialist     1  0
Unicorn Wrangler  0  1

Cell-by-cell relative error:
Sex                      M      F
Occupation                       
Statistician      0.058824  0.000
Economist         0.000000  0.000
Geographer        0.000000  0.125
IT Specialist     0.100000  0.000
Unicorn Wrangler       NaN  0.500

Total L1 Error: 4
Relative L1 Error: 4.00%
